In [72]:
import pandas as pd
import numpy as np

In [73]:
import keras
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential

In [ ]:
## Load crunchbase dataset

In [74]:
crunchbase=pd.read_csv('crunchbase.csv')
crunchbase=crunchbase.set_index("Unnamed: 0")

In [75]:
crunchbase.head()

,company,funding_type,funding_status,money,industry,lead_investors,other_investors,descrip,date,currency,value,usd_value,year
Unnamed: 0,,,,,,,,,,,,,
0,EP Energy Corporation,Private Equity,IPO,"$7,150,000,000","Energy, Oil and Gas",—,"Access Industries, Apollo, Riverstone Holdings",EP Energy produces natural oils and gases whic...,24-May-12,$,7.150000e+09,7.150000e+09,2012
1,Alibaba Group,Private Equity,IPO,"$2,000,000,000","Association, B2B, E-Commerce,,,,",China Investment Corporation,"Boyu Capital, China Investment Corporation,,,,",Alibaba Group enables businesses to transform ...,1-Sep-12,$,2.000000e+09,2.000000e+09,2012
2,Wave,Private Equity,M&A,"$1,050,000,000","Internet, Telecommunications,,",—,"GI Partners, Oak Hill Capital Partners","Wave is a leading provider of video, internet ...",14-Nov-12,$,1.050000e+09,1.050000e+09,2012
3,TOPSHOP TOPMAN,Private Equity,Private Equity,"$805,000,000","E-Commerce, Fashion, Lifestyle",Leonard Green & Partners,Leonard Green & Partners,TOPSHOP TOPMAN is a latest women's fashion and...,6-Dec-12,$,8.050000e+08,8.050000e+08,2012
4,Enel OGK-5,Private Equity,Private Equity,"$625,000,000","Energy, Enterprise, Industrial",Xenon Capital Partners,"Macquarie Group, Russian Direct Investment Fund,",Enel OGK-5 is an Italian power generation comp...,16-Mar-12,$,6.250000e+08,6.250000e+08,2012


In [76]:
np.unique(crunchbase.industry)

array(['3D Printing', '3D Printing, 3D Technology',
       '3D Printing, 3D Technology, Advanced Materials,', ...,
       'mHealth, Personal Health, Wellness', 'mHealth, Wellness', '—'], dtype=object)

In [77]:
industries_list=[x.split(",") for x in crunchbase.industry]

In [78]:
## fix the empties
for i in range(len(industries_list)):
    industries_list[i]=[x for x in industries_list[i] if len(x)>0]
    industries_list[i]=[str(industries_list[i][j])[1:]  
                        if str(industries_list[i][j])[0]==' ' else industries_list[i][j]
                       for j in range(len(industries_list[i]))
                       ]

In [79]:
keys=list(np.unique([x for y in industries_list for x in y]))

In [80]:
keys.remove('—')

In [ ]:
pd.Series(keys).to_csv('keys.csv')

In [19]:
crunchbase[crunchbase.industry=='—'].groupby('year').sum()['usd_value']/10**6

year
2012      16.182637
2013      52.840189
2014      80.188493
2015     116.696377
2016     317.288664
2017     139.724260
2018    1996.292435
2019    1448.323442
2020      12.605356
Name: usd_value, dtype: float64

In [82]:
#top keys

industries_values={}
for key in keys:
    industries_values[key]=np.sum([crunchbase['usd_value'][ind] 
                                   for ind in range(len(industries_list)) if key in industries_list[ind]])/10**6

In [83]:
industries_values=pd.DataFrame([industries_values.keys(),industries_values.values()]).transpose()
industries_values.columns=['industry', 'usd_value']
industries_values=industries_values.sort_values('usd_value', ascending=False)
industries_values=industries_values.set_index('industry')

In [84]:
len(industries_values)

716

In [47]:
top_keys=industries_values.head(70).index

In [49]:
ind=[i for i in range(len(industries_list)) if len(set(top_keys).intersection(set(industries_list[i])))>0]

In [52]:
len(ind)/crunchbase.shape[0]

0.8754197872609496

In [54]:
np.sum(crunchbase.loc[ind]['usd_value']/10**6)/np.sum(crunchbase['usd_value']/10**6)

0.9124331121652371

## Prepare the train, dev and test sets

In [63]:
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
import re
from string import punctuation
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vnuretdi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [64]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vnuretdi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [150]:
crunchbase

,company,funding_type,funding_status,money,industry,lead_investors,other_investors,descrip,date,currency,value,usd_value,year
Unnamed: 0,,,,,,,,,,,,,
0,EP Energy Corporation,Private Equity,IPO,"$7,150,000,000","Energy, Oil and Gas",—,"Access Industries, Apollo, Riverstone Holdings",EP Energy produces natural oils and gases whic...,24-May-12,$,7.150000e+09,7.150000e+09,2012
1,Alibaba Group,Private Equity,IPO,"$2,000,000,000","Association, B2B, E-Commerce,,,,",China Investment Corporation,"Boyu Capital, China Investment Corporation,,,,",Alibaba Group enables businesses to transform ...,1-Sep-12,$,2.000000e+09,2.000000e+09,2012
2,Wave,Private Equity,M&A,"$1,050,000,000","Internet, Telecommunications,,",—,"GI Partners, Oak Hill Capital Partners","Wave is a leading provider of video, internet ...",14-Nov-12,$,1.050000e+09,1.050000e+09,2012
3,TOPSHOP TOPMAN,Private Equity,Private Equity,"$805,000,000","E-Commerce, Fashion, Lifestyle",Leonard Green & Partners,Leonard Green & Partners,TOPSHOP TOPMAN is a latest women's fashion and...,6-Dec-12,$,8.050000e+08,8.050000e+08,2012
4,Enel OGK-5,Private Equity,Private Equity,"$625,000,000","Energy, Enterprise, Industrial",Xenon Capital Partners,"Macquarie Group, Russian Direct Investment Fund,",Enel OGK-5 is an Italian power generation comp...,16-Mar-12,$,6.250000e+08,6.250000e+08,2012
5,"Capistrano Wind Partners, LLC",Private Equity,Private Equity,"$460,000,000",Energy,Teachers Insurance and Annuity Association Col...,"Cook Inlet Region, Incorporated,",Entity created to manage several operating win...,28-Feb-12,$,4.600000e+08,4.600000e+08,2012
6,Legendary Entertainment,Private Equity,M&A,"$443,000,000","Content Delivery Network, Film,,,",Waddell & Reed,Waddell & Reed,"Legendary Entertainment offers film, televisio...",18-Dec-12,$,4.430000e+08,4.430000e+08,2012
7,GreatPoint Energy,Series D,Late Stage Venture,"$420,000,000","Clean Energy, Energy, Information Technology",Wanxiang America,Wanxiang America,GreatPoint Energy is a technology-driven natur...,17-Feb-12,$,4.200000e+08,4.200000e+08,2012
8,Fisker Automotive,Private Equity,M&A,"$392,000,000","Automotive, Autonomous Vehicles,,,,",—,Advanced Equities,Fisker Automotive was an American company know...,2-Apr-12,$,3.920000e+08,3.920000e+08,2012


In [151]:
dataset=crunchbase[['descrip','industry', 'company']]

In [152]:
train=dataset[dataset.industry!='—']
test=dataset[dataset.industry=='—']

In [153]:
train.head()

,descrip,industry,company
Unnamed: 0,,,
0,EP Energy produces natural oils and gases whic...,"Energy, Oil and Gas",EP Energy Corporation
1,Alibaba Group enables businesses to transform ...,"Association, B2B, E-Commerce,,,,",Alibaba Group
2,"Wave is a leading provider of video, internet ...","Internet, Telecommunications,,",Wave
3,TOPSHOP TOPMAN is a latest women's fashion and...,"E-Commerce, Fashion, Lifestyle",TOPSHOP TOPMAN
4,Enel OGK-5 is an Italian power generation comp...,"Energy, Enterprise, Industrial",Enel OGK-5


## Data preprocessing to models input

In [99]:
import keras
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential

In [100]:
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D
from keras.optimizers import Adam

In [93]:
train.head()

,descrip,industry,words,lemma
Unnamed: 0,,,,
0,EP Energy produces natural oils and gases whic...,"Energy, Oil and Gas","[ep, energy, produces, natural, oils, and, gas...","[ep, energy, produce, natural, oil, and, gas, ..."
1,Alibaba Group enables businesses to transform ...,"Association, B2B, E-Commerce,,,,","[alibaba, group, enables, businesses, to, tran...","[alibaba, group, enables, business, to, transf..."
2,"Wave is a leading provider of video, internet ...","Internet, Telecommunications,,","[wave, is, a, leading, provider, of, video, ,,...","[wave, is, a, leading, provider, of, video, ,,..."
3,TOPSHOP TOPMAN is a latest women's fashion and...,"E-Commerce, Fashion, Lifestyle","[topshop, topman, is, a, latest, women, 's, fa...","[topshop, topman, is, a, latest, woman, 's, fa..."
4,Enel OGK-5 is an Italian power generation comp...,"Energy, Enterprise, Industrial","[enel, ogk-5, is, an, italian, power, generati...","[enel, ogk-5, is, an, italian, power, generati..."


In [154]:
train['words']=pd.Series([word_tokenize(x.lower()) for x in train['descrip']], index=train.index)
test['words']=pd.Series([word_tokenize(x.lower()) for x in test['descrip']], index=test.index)

C:\Users\vnuretdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\vnuretdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [166]:
def tokenize(x):
    try:
        return (word_tokenize(x.lower()))
    except:
        return []

In [167]:
train['title']=pd.Series([tokenize(x) for x in train['company']], index=train.index)
test['title']=pd.Series([tokenize(x) for x in test['company']], index=test.index)

C:\Users\vnuretdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\vnuretdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [192]:
def cut_title(train, i):
    try:
        return [train['words'][i][s] for s in range(len(train['title'][i])) 
                 if train['words'][i][s]!=train['title'][i][s]]+train['words'][i][len(train['title'][i]):]
    except: return train['words'][i]

In [193]:
train['words']=pd.Series([cut_title(train, i) for i in train.index],
                        index=train.index)

C:\Users\vnuretdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [194]:
test['words']=pd.Series([cut_title(test, i) for i in test.index],
                        index=test.index)

C:\Users\vnuretdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [195]:
train['lemma']=pd.Series([[WordNetLemmatizer().lemmatize(w) for w in x] for x in train['words']], index=train.index)
test['lemma']=pd.Series([[WordNetLemmatizer().lemmatize(w) for w in x] for x in test['words'] ], index=test.index)

C:\Users\vnuretdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\vnuretdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Probability model based on Bernoulli distribution

In [254]:
from sklearn.datasets import fetch_mldata
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [260]:
def X_onehot(X_train, i):
    return [1 if x in X_train[i] else 0 for x in range(V)]

In [264]:
X_train_binarized=np.asarray([X_onehot(X_train, i) for i in range(X_train.shape[0])])

In [267]:
X_dev_binarized=np.asarray([X_onehot(X_dev, i) for i in range(X_dev.shape[0])])

In [268]:
nb = BernoulliNB(alpha=1)

In [275]:
X_train_binarized[0]

array([1, 1, 1, ..., 0, 0, 0])

In [291]:
def return_label(y_train, i):
    try: return list(y_train[i]).index(1)
    except: return len(y_train[0])

In [292]:
train_labels=np.asarray([return_label(y_train, i) for i in range (len(y_train))])

In [294]:
dev_labels=np.asarray([return_label(y_dev, i) for i in range (len(y_dev))])

In [295]:
nb.fit(X_train_binarized, train_labels)

BernoulliNB(alpha=1, binarize=0.0, class_prior=None, fit_prior=True)

In [297]:
preds = nb.predict(X_dev_binarized)

In [298]:
acc=np.mean(dev_labels == preds)

print ("accuracy: {:4}".format(acc))

accuracy: 0.34335674223500023


## RNN

In [197]:
vocab=set([w for x in train['lemma'] for w in x])
V=len(vocab)
max_len=max([len(x) for x in train['words']])

In [198]:
max_features = V
max_words = max_len
batch_size = 128
epochs = 10
num_classes=len(top_keys)

In [199]:
from sklearn.model_selection import train_test_split

traindf, devdf  = train_test_split(train,  test_size=0.3)

In [200]:
X_train=traindf['lemma']
X_dev=devdf['lemma']
X_test=test['lemma']

In [201]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))

In [202]:
X_train = tokenizer.texts_to_sequences(X_train)
X_dev = tokenizer.texts_to_sequences(X_dev)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_dev = sequence.pad_sequences(X_dev, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
print(X_train.shape,X_dev.shape,X_test.shape)

(49504, 110) (21217, 110) (446, 110)


In [203]:
def to_onehot(i):
    return [0 if x not in industries_list[i] else 1 for x in top_keys]

In [204]:
y_train=[to_onehot(i) for i in traindf.index]
y_train=np.asarray(y_train)

In [205]:
y_dev=[to_onehot(i) for i in devdf.index]
y_dev=np.asarray(y_dev)

In [206]:
model1=Sequential()
model1.add(Embedding(max_features,100,mask_zero=True))
model1.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model1.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model1.add(Dense(num_classes,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 100)         2327700   
_________________________________________________________________
lstm_7 (LSTM)                (None, None, 64)          42240     
_________________________________________________________________
lstm_8 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_4 (Dense)              (None, 70)                2310      
Total params: 2,384,666
Trainable params: 2,384,666
Non-trainable params: 0
_________________________________________________________________


In [207]:
history1=model1.fit(X_train, y_train, validation_data=(X_dev, y_dev),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 49504 samples, validate on 21217 samples
Epoch 1/10
49504/49504 [==============================] - 217s 4ms/step - loss: 6.0324 - acc: 0.1406 - val_loss: 5.7079 - val_acc: 0.2575
Epoch 2/10
49504/49504 [==============================] - 202s 4ms/step - loss: 5.2250 - acc: 0.2798 - val_loss: 4.9315 - val_acc: 0.2994
Epoch 3/10
49504/49504 [==============================] - 204s 4ms/step - loss: 4.7801 - acc: 0.2930 - val_loss: 4.6579 - val_acc: 0.3039
Epoch 4/10
49504/49504 [==============================] - 204s 4ms/step - loss: 4.5076 - acc: 0.3098 - val_loss: 4.4537 - val_acc: 0.3238
Epoch 5/10
49504/49504 [==============================] - 196s 4ms/step - loss: 4.2743 - acc: 0.3469 - val_loss: 4.2828 - val_acc: 0.3576
Epoch 6/10
49504/49504 [==============================] - 200s 4ms/step - loss: 4.0773 - acc: 0.3739 - val_loss: 4.1623 - val_acc: 0.3742
Epoch 7/10
49504/49504 [==============================] - 201s 4ms/step - loss: 3.9365 - acc: 0.3932 - val_loss: 4.0943 - 

In [138]:
from keras.models import model_from_json

def save_model(model,name):
    '''
    save model architecture and model weights
    '''
    json_string = model.to_json()
    open(name+'_architecture.json', 'w').write(json_string)
    model.save_weights(name+'_weights.h5')
    
def load_model(name):
    model = model_from_json(open(name+'_architecture.json').read())
    model.load_weights(name + '_weights.h5')
    return(model)

In [139]:
save_model(model1,"model")

In [208]:
save_model(model1,"model1")

In [209]:
y_pred1=model1.predict_classes(X_test,verbose=1)

446/446 [==============================] - 1s 2ms/step


In [144]:
y_pred1

array([ 5,  4,  5,  5, 59,  0,  5, 29, 17,  1,  4,  5, 14,  3, 30,  8, 40,
       14,  8, 50,  8, 63,  5,  4, 49,  5,  0,  3, 50, 14, 14, 27,  5,  7,
        8,  5, 17,  8,  8,  5,  5, 18, 14,  8,  8, 48,  1,  7, 10, 49,  8,
        9,  1,  3,  8,  4, 59,  8,  7, 29, 24,  4,  8, 29,  8,  8,  0, 27,
        2,  8,  2, 25,  5, 14,  3, 12,  4, 59, 49,  4, 29,  7, 23, 12, 29,
       29,  0,  4,  4, 40, 37,  5,  5, 51, 24,  0,  5, 35,  8, 24,  3, 10,
       23,  3,  3,  8, 59,  5, 45, 24,  7, 24,  3,  3, 14,  5, 10,  5, 40,
       17,  3, 40,  1, 12,  5,  5,  5, 40,  5,  5,  2,  5, 14,  7,  4,  8,
        0,  9,  3,  2,  2,  8, 18, 12, 16,  4,  5, 39,  7,  1,  5,  2,  5,
       19,  3,  7,  5,  4,  5,  5, 17,  5, 14,  2,  5, 48,  5, 29,  3, 35,
       13,  5, 29, 14,  4, 40,  3,  4, 12,  0,  5, 40,  0,  8,  7, 21,  2,
       16,  3,  4,  0, 59,  8, 13, 40,  3,  5,  2,  6,  4, 25, 24, 39, 24,
       10,  3,  0, 16,  4,  3,  8,  3,  0,  4,  1, 16, 24,  3,  8,  1,  2,
        4, 17,  2,  4, 17

In [210]:
y_pred1

array([ 5,  4,  0, 23,  8,  5,  5, 29,  3,  5, 29,  3,  8,  0,  4,  4, 10,
        1, 24, 29, 14, 14,  0,  5,  5,  5,  5,  0,  5,  8,  0,  4,  5, 14,
        5,  5,  5,  8,  7, 10,  4,  5, 48,  5, 49,  5,  4,  5,  5,  7,  1,
        5, 36,  8, 59,  8,  5,  0, 27,  2,  5,  1, 24,  0, 14,  7, 10,  4,
        2,  4,  4, 29,  7, 45, 10, 29, 59, 27,  4,  4, 40,  0, 10,  5, 23,
        5,  0,  5, 40,  5,  8,  5, 10,  5,  5,  3,  5, 19,  5, 45,  5,  7,
        5,  0, 24,  3,  5, 17, 63,  5, 17,  5,  0,  5,  8,  5,  5,  5,  0,
        8, 39,  2, 43, 14,  7,  4,  5,  0,  9,  5,  1,  9,  5,  4, 10, 36,
        4,  5, 39,  7,  2,  0,  2, 14,  5,  5,  7,  5,  4,  5,  1, 17, 10,
       14,  1,  5, 24,  9, 29,  3, 40,  3,  7,  1,  0,  8, 40,  0,  4,  0,
        0,  5,  5,  5,  5,  4, 21,  5, 18,  5,  4,  1,  3,  5,  5, 29,  4,
        1,  2, 35,  4,  8, 17, 17, 10,  5, 24,  0,  4,  4,  0,  5,  5,  5,
        7,  1,  4,  5,  8,  5,  1, 14,  4, 17,  2, 36, 18,  1,  2, 40,  2,
        0,  5, 48, 48, 27

In [211]:
pd.concat([pd.Series([top_keys[x] for x in y_pred1]), pd.Series(list(test['descrip']))], axis=1)

,0,1
0,Information Technology,Ui Link
1,Financial Services,Investment manager within energy infrastructure
2,E-Commerce,"Everything, everywhere."
3,Cloud Computing,Litigation Support
4,Automotive,AquaJet Ltd. has been established to exploit a...
5,Information Technology,An industry leader in non-hazardous recycling ...
6,Information Technology,Newfo
7,Manufacturing,We are trying to preserve the natural characte...
8,Internet,Closed
9,Information Technology,Genestamp


## CNN

In [236]:
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, Concatenate

In [243]:
from keras.layers import Input, merge

In [245]:
from keras.layers import Embedding, Input
from keras.layers.merge import concatenate
from keras.models import Model

activation='relu'
    
# Defining input 1
input1=Input(shape=(max_words, ))
x1 = Embedding(input_dim=max_features, output_dim=100)(input1)
x1 = Dropout(0.4)(x1)
x1 = Conv1D(padding="same", activation="relu", strides=1, filters=64, kernel_size=2)(x1)
x1 = GlobalMaxPooling1D()(x1)

# Defining input 2
input2=Input(shape=(max_words,))
x2 = Embedding(input_dim=max_features, output_dim=100)(input2)
x2 = Dropout(0.4)(x2)
x2 = Conv1D(padding="same", activation="relu", strides=1, filters=64, kernel_size=3)(x2)
x2 = GlobalMaxPooling1D()(x2)

    # Defining input 3
input3 = Input(shape=(max_words,))
x3 = Embedding(input_dim=max_features, output_dim=100)(input3)
x3 = Dropout(0.4)(x3)
x3 = Conv1D(padding="same", activation="relu", strides=1, filters=64, kernel_size=4)(x3)
x3 = GlobalMaxPooling1D()(x3)
    
    # Merging subnetworks
#inputs = merge([input1, input2, input3],  mode="concat") 
inputs = merge([x1, x2, x3],  mode="concat") 

C:\Users\vnuretdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Users\vnuretdi\Anaconda3\lib\site-packages\keras\legacy\layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [246]:
outputs = Dense(32, activation='relu')(inputs)
outputs = Dropout(0.2)(outputs)
outputs = Dense(num_classes, activation='softmax')(outputs)

In [247]:
 # Final Dense layer and compilation

model = Model(inputs=[input1, input2, input3], outputs=outputs)
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])

In [248]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 110)          0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 110)          0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 110)          0                                            
__________________________________________________________________________________________________
embedding_24 (Embedding)        (None, 110, 100)     2327700     input_4[0][0]                    
__________________________________________________________________________________________________
embedding_

In [249]:
model.fit([X_train, X_train, X_train], y_train, validation_data=([X_dev, X_dev, X_dev], y_dev), epochs=epochs, batch_size=batch_size, verbose=1)

Train on 49504 samples, validate on 21217 samples
Epoch 1/10
49504/49504 [==============================] - 207s 4ms/step - loss: 5.5730 - acc: 0.2257 - val_loss: 4.5559 - val_acc: 0.3642
Epoch 2/10
49504/49504 [==============================] - 239s 5ms/step - loss: 4.3868 - acc: 0.3472 - val_loss: 4.0564 - val_acc: 0.3839
Epoch 3/10
49504/49504 [==============================] - 234s 5ms/step - loss: 3.9826 - acc: 0.3817 - val_loss: 3.8510 - val_acc: 0.4061
Epoch 4/10
49504/49504 [==============================] - 229s 5ms/step - loss: 3.7174 - acc: 0.4089 - val_loss: 3.7099 - val_acc: 0.4126
Epoch 5/10
49504/49504 [==============================] - 223s 4ms/step - loss: 3.5131 - acc: 0.4225 - val_loss: 3.6316 - val_acc: 0.4191
Epoch 6/10
49504/49504 [==============================] - 224s 5ms/step - loss: 3.3578 - acc: 0.4341 - val_loss: 3.5771 - val_acc: 0.4221
Epoch 7/10
49504/49504 [==============================] - 200s 4ms/step - loss: 3.2180 - acc: 0.4453 - val_loss: 3.5391 - 

In [253]:
save_model(model,"cnn_model")

In [301]:
cnn_model=load_model('cnn_model')

C:\Users\vnuretdi\Anaconda3\lib\site-packages\keras\engine\topology.py:1271: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [323]:
y_pred=cnn_model.predict([X_dev, X_dev, X_dev])

In [328]:
y_pred=[np.argmax(x) for x in y_pred]

In [352]:
dev_result=pd.concat([pd.Series([list(dev_labels).count(i) for i in range(len(top_keys)+1)]),
    pd.Series([(len([j for j in range(len(dev_labels)) if (dev_labels[j]==i)&(y_pred[j]==i)])/list(dev_labels).count(i)) if 
               list(dev_labels).count(i)>0 else 0
               for i in range(len(top_keys)+1)]),
    pd.Series([len([j for j in range(len(dev_labels)) if (dev_labels[j]!=i)&(y_pred[j]==i)])/list(dev_labels).count(i) if 
               list(dev_labels).count(i)>0 else 0
               for i in range(len(top_keys)+1)]),
    pd.Series([len([j for j in range(len(dev_labels)) if (dev_labels[j]==i)&(y_pred[j]!=i)])/list(dev_labels).count(i) if 
               list(dev_labels).count(i)>0 else 0
               for i in range(len(top_keys)+1)])], axis=1)
dev_result.columns=['count', '% correct', '%false positive', '%false negative']

In [355]:
dev_result.index=list(top_keys)+['Other']

In [356]:
dev_result.head(20)

,count,% correct,%false positive,%false negative
E-Commerce,1941,0.650180,0.446162,0.349820
Health Care,2297,0.568568,0.235525,0.431432
Biotechnology,913,0.646221,0.553122,0.353779
Internet,1085,0.276498,0.689401,0.723502
Financial Services,967,0.674250,0.541882,0.325750
Information Technology,1137,0.204925,0.497801,0.795075
Finance,288,0.184028,0.541667,0.815972
Artificial Intelligence,1277,0.672670,0.443226,0.327330
Automotive,334,0.517964,0.649701,0.482036
Energy,196,0.658163,0.627551,0.341837


### Save test results

In [359]:
y_test=cnn_model.predict([X_test, X_test, X_test])
y_test=[np.argmax(x) for x in y_test]

In [363]:
labels=list(top_keys)+["Other"]
y_test=[labels[i] for i in y_test]

In [369]:
crunchbase['train']=pd.Series([0]*crunchbase.shape[0])
crunchbase['train']=pd.Series([1 if crunchbase.loc[i]['industry']=='—' else 0 for
                                                i in crunchbase.index], index=crunchbase.index)

In [396]:
for i, index in enumerate(crunchbase[crunchbase.train==1].index):
    crunchbase.loc[index, 'industry']=y_test[i]

In [398]:
crunchbase.to_csv('crunchbase_complete.csv')

In [402]:
crunchbase=pd.read_csv('crunchbase_complete.csv').iloc[:,1:]

In [406]:
crunchbase[crunchbase.train==1].head()

,company,funding_type,funding_status,money,industry,lead_investors,other_investors,descrip,date,currency,value,usd_value,year,train
44510,Chaodian Culture,Series A,Early Stage Venture,"CN¥38,200,000",Health Care,—,Qiming Venture Partners,Chaodian Culture,1-Dec-18,CN¥,38200000.0,5499750.307,2018,1
47167,Bomming Vision,Series A,Early Stage Venture,"CN¥30,000,000",Manufacturing,Qiming Venture Partners,Qiming Venture Partners,Zhuhai Bomming Vision Technology is a high-tec...,1-Apr-17,CN¥,30000000.0,4355259.227,2017,1
47174,editorji,Seed,Seed,"?300,000,000",Media and Entertainment,—,"Airtel, HT Media",Short Video News Platform,24-Aug-18,?,300000000.0,4291242.601,2018,1
47435,Lvjun Risk Control,Series A,Early Stage Venture,"CN¥30,000,000",Information Technology,Sichuan E-commerce Industry Equity Investment ...,Sichuan E-commerce Industry Equity Investment ...,Lvjun Risk Control is a risk management contro...,12-Dec-19,CN¥,30000000.0,4263020.900,2019,1
47466,Sanfu Interactive Entertainment,Series A,Early Stage Venture,"CN¥30,000,000",Media and Entertainment,Shenzhen Guozhong Venture Capital Management,Shenzhen Guozhong Venture Capital Management,Sanfu Interactive Entertainment is an engages ...,2-Sep-19,CN¥,30000000.0,4183225.905,2019,1


In [408]:
ind=44510

In [418]:
crunchbase[(crunchbase.train==2)&(crunchbase.company==crunchbase.loc[ind, 'company'])]

,company,funding_type,funding_status,money,industry,lead_investors,other_investors,descrip,date,currency,value,usd_value,year,train
13701,Chaodian Culture,Series B,Early Stage Venture,"$150,000,000",Financial Services,—,Qiming Venture Partners,Chaodian Culture,1-Dec-19,$,150000000.0,1.500000e+08,2019,2
33077,Chaodian Culture,Series A,Early Stage Venture,"CN¥79,430,000",Financial Services,—,Qiming Venture Partners,Chaodian Culture,1-Apr-18,CN¥,79430000.0,1.263305e+07,2018,2


In [419]:
for ind in crunchbase[crunchbase.train==1].index:
    if crunchbase.loc[ind, 'company'] in list(crunchbase[crunchbase.train==2].company):
        j=crunchbase[(crunchbase.train==2)&(crunchbase.company==crunchbase.loc[ind, 'company'])].index[0]
        crunchbase.loc[ind, 'industry']=crunchbase.loc[j, 'industry']
        

In [421]:
crunchbase.to_csv('crunchbase.csv')